In [ ]:
import numpy as np

### Exercise 6.1.1:
Suppose there are 100 items, numbered 1 to 100, and also 100 baskets, also numbered 1 to 100. Item `i` is in basket `b` if and only if `i` divides `b` with no remainder. Thus, item 1 is in all the baskets, item 2 is in all fifty of the even-numbered baskets, and so on. Basket 12 consists of items {1,2,3,4,6,12}, since these are all the integers that divide 12. Answer the following questions:

(a) If the support threshold is 5, which items are frequent?

(b) If the support threshold is 5, which pairs of items are frequent?

(c) What is the sum of the sizes of all the baskets?

#### Solution:

(a) Item `i` is in basket `b` if `i` is a factor of `b`. In other words, `i` is in basket `b` if and only if there exists a constant integer `k`>=1 such that `b=k*i`. As a result, item `i` is found in 5 or more baskets if `100/i >=5`. Therefore items {1},{2},...,{20} represent the frequent singletons.

(b) To find the pairs `(i,j)` that are frequent, we make use of the following two facts: 

1. If both `i` and `j` are factors of `b` and `(i*j)<=b`, then `i*j` is also a factor of `b`. (**Proof:** Since `i` is a factor of `b`, there exists a constant `k1` such that `b=k1*i`. Furthermore, our assumption of `(i*j)<=b` coupled with the fact that `j` is also a factor of `b` necessarily implies that `k1=k2*j` for some `k2>=1`. As a result, `b=k2*(j*i)`.) Hence, if `i`, `j`, and `(i*j)` are all frequent singletons, then the pair (`i,j`) is a frequent doubleton.

2. If `j` is frequent and `j mod i = 0`, then the support of `j` is a subset of the support of `i`. As a result `i` must be in every basket that `j` is in, and so the pair `(i,j)` is frequent.

It follows from facts 1. and 2. that a pair `(i,j)` (for `i<j=1,2,..,20`) is frequent if either of the conditions [`i*j<=20`] or [`j mod i = 0`] hold. The frequent pairs are:
- `(1,j)` for `j=2,3,...,20`
- `(2,j)` for `j=3,4,...,10` and `j=12,14,16,18,20`
- `(3,j)` for `j=4,5,6` and `j=9,12,15,18`
- `(4,j)` for `j=5` and `j=8,12,16,20`
- `(5,j)` for `j=10,15,20`
- `(6,j)` for `j=12,18`
- `(7,14)`
- `(8,16)`
- `(9,18)`
- `(10,20)`

(c) Define, `num_factors(b)` as the number of factors that b has. Then sum of the sizes all baskets = `sum(num_factors(b), b=1,2,...,20)`.

So, I didn't feel like thinking about how to grab prime factors of a number myself, so I stackoverflow'ed this. [Here](http://stackoverflow.com/questions/16996217/prime-factorization-list) is simple function to extract the prime factors of a list (it essentially follows how one would find prime factors by hand). 

In [21]:
def primes(n):
    primfac = []
    d = 2
    while d*d <= n:
        while (n % d) == 0:
            primfac.append(d)  # supposing you want multiple factors repeated
            n /= d
        d += 1
    if n > 1:
        primfac.append(n)
    return primfac

In [48]:
# create dictionaries for prime factors of baskets 1,2,...,100
primefactors = {}
for b in range(1,101):
    # initializing the dictionary for each basket b
    primefactors[b] = {fac:0 for fac in primes(b)}
    for key in primes(b):
        primefactors[b][key] += 1

In [53]:
# for example, the prime factorization of 12 = 2^2 * 3
primefactors[12]

{2: 2, 3: 1}

To get the number of factors of 12, we add 1 to each of the replications of its factors and apply the multiplication rule. So 12 has (2+1)*(1+1)=6 factors in total.

In [61]:
# to get number of factors, we add 1 to each rep of factor and apply multiplication rule
def num_factors(b):
    numfac = 1
    for fac,reps in primefactors[b].items():
        numfac *= reps + 1
    return numfac

In [62]:
num_factors(12)

6

In [68]:
# sum of the sizes of all baskets
sizeofbaskets = [num_factors(b) for b in range(1,101)]
totalsize = sum(sizeofbaskets)

In [69]:
totalsize

482

### Exercise 6.1.2
For the item-basket data of Exercise 6.1.1, which basket is the largest?

In [70]:
# the largest baskets hav 12 items
max(sizeofbaskets)

12

In [71]:
# these baskets are
for b in range(1,101):
    if num_factors(b) == 12:
        print b

60
72
84
90
96


### Exercise 6.1.3
Suppose there are 100 items, numbered 1 to 100, and also 100 baskets, also numbered 1 to 100. Item `i` is in basket `b` if and only if `b` divides `i` with no remainder. For example, basket 12 consists of items {12,24,36,48,60,72,84,96}. Repeat Exercise 6.1.1 for this data.

#### Solution

(a) Basket `b` consists of items which are multiples of `b`. Alternatively, item `i` is in basket `b` if `b` is a factor of `i`. Thus, item `i` is frequent if it has at least 5 factors.

In [73]:
# List of frequent items
L1 = [b for b in range(1,101) if num_factors(b)>=5]

In [75]:
for b in L1:
    print b,

12 16 18 20 24 28 30 32 36 40 42 44 45 48 50 52 54 56 60 63 64 66 68 70 72 75 76 78 80 81 84 88 90 92 96 98 99 100


(b) Clearly, `(i,j)` represent a frequent pair if `i` and `j` share at least 5 common factors.

In [181]:
def lexorders_ofexp(b):
    """
    This function returns the lexicographic ordering of the exponents
    of the prime factors of b. We can use this to get all the factors
    of b.
    """
    n = len(primefactors[b])
    ati = primefactors[b].values()
    foo = []
    if n == 1:
        for j in range(ati[0]+1):
            foo.append([j])
    if n == 2:
        i = 0
        while i < ati[0]+1:
            j = 0
            while j < ati[1]+1:
                foo.append([i,j])
                j+=1
            i+=1
    if n == 3:
        i = 0
        while i < ati[0]+1:
            j = 0
            while j < ati[1]+1:
                k = 0
                while k < ati[2]+1:
                    foo.append([i,j,k])
                    k+=1
                j+=1
            i+=1
    return foo        

In [182]:
# getting all factors of b from its prime factors
def factors(b):
    facs = []
    exps = lexorders_ofexp(b)
    for i in range(num_factors(b)):
        bar = 1 # this also takes care of base case factors(1)
        for el,key in enumerate(primefactors[b].keys()):
            bar *= key**exps[i][el]
        facs.append(bar)
    return facs

In [198]:
def num_commonfactors(b1,b2):
    foo = factors(b1)
    bar = factors(b2)
    return len([el for el in factors(b1) if el in factors(b2)])

In [199]:
factors(72)

[1, 3, 9, 2, 6, 18, 4, 12, 36, 8, 24, 72]

In [200]:
factors(12)

[1, 3, 2, 6, 4, 12]

In [202]:
# common factors
[el for el in factors(12) if el in factors(72)]

[1, 3, 2, 6, 4, 12]

In [203]:
num_commonfactors(12,72)

6

#### We are now ready to grab the frequent pairs.

In [210]:
# List of pairs (i,j) with at least 5 common factors
for i in range(1,100):
    for j in range(i+1,101):
        if num_commonfactors(i,j) >= 5:
            print (i,j),

(12, 24) (12, 36) (12, 48) (12, 60) (12, 72) (12, 84) (12, 96) (16, 32) (16, 48) (16, 64) (16, 80) (16, 96) (18, 36) (18, 54) (18, 72) (18, 90) (20, 40) (20, 60) (20, 80) (20, 100) (24, 36) (24, 48) (24, 60) (24, 72) (24, 84) (24, 96) (28, 56) (28, 84) (30, 60) (30, 90) (32, 48) (32, 64) (32, 80) (32, 96) (36, 48) (36, 54) (36, 60) (36, 72) (36, 84) (36, 90) (36, 96) (40, 60) (40, 80) (40, 100) (42, 84) (44, 88) (45, 90) (48, 60) (48, 64) (48, 72) (48, 80) (48, 84) (48, 96) (50, 100) (54, 72) (54, 90) (56, 84) (60, 72) (60, 80) (60, 84) (60, 90) (60, 96) (60, 100) (64, 80) (64, 96) (72, 84) (72, 90) (72, 96) (80, 96) (80, 100) (84, 96)


(c) The size of basket `b` is `floor(100/b)`

In [215]:
sizeofbaskets = [int(100/b) for b in range(1,101)]
totalsize = sum(sizeofbaskets)
totalsize

482